In [ ]:
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha.neo
from Artha.neo import Neo
import sqlite3
from tqdm import tqdm
import csv


def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = str(row[idx])
    return d

neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
conn = sqlite3.connect("../data/ucheckra_.db")
conn.row_factory = dict_factory
checkra_follows = conn.cursor().execute("select * from following").fetchall()
path = "/Users/harshasomisetty/Library/Application Support/com.Neo4j.Relate/Data/dbmss/dbms-8c0b6b9b-8405-42f7-aee7-4bff9bf1d898/import/follows.csv"
print("done")

In [ ]:
print(len(neo.get_nodes()), len(neo.get_relations()))

In [ ]:
for record in neo.get_nodes():
    print(record["username"])

In [ ]:
neo.clear_db()

In [ ]:
def user_edges(username, location = "../data/users"):
    conn_temp = sqlite3.connect(location+f"/u{username}.db")
    conn_temp.row_factory = dict_factory

    try:
        temp_follows = conn_temp.cursor().execute("select * from following").fetchall()
    except:
        return -1
    else:
        for i in temp_follows:
            i["followed_by"] = username
        with open(path, "w+") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames = ["id", "name", "username", "followed_by"])
            writer.writeheader()
            writer.writerows(temp_follows)
            if username == "checkra_":
                neo.session.run(
                        "Merge (n:Person {username: 'checkra_'})"
                        "ON CREATE SET n.id ='1356259499431129092', n.name = 'Checkra', n.username = 'checkra_'")
            result = neo.session.run(
                        "LOAD CSV WITH HEADERS FROM 'file:/follows.csv' AS row "
                        "MERGE (n:Person {id:row.id}) "
                        "ON CREATE SET n.id = row.id, n.name=row.name, n.username=row.username "
                        "return n"
                        # "WITH n, row "
                        # "Match (m:Person {username: row.followed_by}) "
                        # "MERGE (m)-[:FOLLOWS]->(n) "
                        # "return n, m"
                        )
            

        # neo.session.run("USING PERIODIC COMMIT "
        #                 "LOAD CSV WITH HEADERS FROM 'file:/follows.csv' AS row "
        #                 "MATCH (n:Person), (m:Person) "
        #                 "WHERE n.username = $username "
        #                 "AND m.username = row.name "
        #                 "MERGE (n)-[:FOLLOWS]->(m)",
        #                 # "RETURN n.username, m.username",
        #                 # "MERGE (n:Person {name: $username})-[:FOLLOWS]->(m:Person {name:row.name})",
        #                 username = username
        #                 )

        return result

# num = user_edges("checkra_", location = "../data")
num = user_edges("Ninjascalp")
records = [record for record in num]
print("done", len(records))

In [ ]:
for record in records:
    print(record["n"]["name"], record["m"]["name"], )

In [ ]:
conn_temp = sqlite3.connect("../data/users/uBrianRoemmele.db")
conn_temp.row_factory = dict_factory


temp_follows = conn_temp.cursor().execute("select id, name, username from following").fetchall()

    # with open(path, "w+") as csvfile:
    #     writer = csv.DictWriter(csvfile, fieldnames = ["id", "name", "username", "by"])
    #     writer.writeheader()
    #     writer.writerows(temp_follows)

In [ ]:
%%time
temp_follows = conn_temp.cursor().execute("select id, name, username from following").fetchall()


    

In [ ]:
%%time

num = user_edges("Ninjascalp")
print("done", "Ninjascalp", num)
# for user in tqdm(checkra_follows):
#     num = user_edges(user["username"])
#     print("done", user["username"], num)

In [ ]:
checkra_follows[:15]

In [ ]:
import os
os.listdir()

In [ ]:
os.listdir("/Users/harshasomisetty/")

In [ ]:
os.getcwd()